In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [19]:
import random

def specialModulus(a):
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    return 6*8^ds2*3^ds3

def RamanujanCongruenceSeries_test(n): # Ramanujan Revisted, p. 290
    ans=0
    randoms=[]
    for k in [1..n+1]:
        sm=specialModulus(k)
        r=ZZ(random.randint(-50, 50))
        randoms=randoms+[r]
        term=r*sm+rmnjntau(k)
        ans=ans+term*x^k
    print("randoms:");print();print(randoms)
    return ans

for a in [1..3]:
     print(RamanujanCongruenceSeries_test(3))

randoms:

[29, -13, 24, -13]
-7088*x^4 + 27900*x^3 - 5640*x^2 + 4177*x
randoms:

[-26, -32, 34, 0]
-1472*x^4 + 39420*x^3 - 13848*x^2 - 3743*x
randoms:

[37, -34, 37, 41]
16240*x^4 + 42876*x^3 - 14712*x^2 + 5329*x


In [4]:
L=LaurentSeriesRing(QQ,'x', default_prec=1050)
import random
import time
def specialModulus(a):
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    return 6*8^ds2*3^ds3

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    randoms=[]
    for k in [1..n+2]:
        randoms=randoms+[ZZ(random.randint(-50, 50))]
    for k in [1..n+1]:
        sm=specialModulus(n)# n, not k.
        term=randoms[k]*sm+rmnjntau(k) # k, not n.
        ans=ans+term*x^k
    return ans

def recip(n):
    return L(1/RamanujanCongruenceSeries(n)).O(n+1)

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]

    
no=[]
for trial in [1..10]:
    zs2=[]
    zs3=[]
    nzs2=[]
    nzs3=[]
    trialno2=[]
    trialno3=[]
    start=time.time()
    for n in [1..100]:
        c=recipPowerConstantTerm(n)
        if c==0:
            zs2=zs2+[mod(n,2)]
            zs3=zs3+[(mod(n,3))]
        if c!=0:
            nzs2=nzs2+[mod(n,2)]
            nzs3=nzs3+[mod(n,3)]
            o2=order(2,c)
            sd2=digitsum(2,n)
            nzs2=nzs2+[mod(n,2)]
            o3=order(3,c)
            sd3=digitsum(3,n)
            test2=3*sd2-o2
            test3=sd3-o3
            if test2!=0:
                no=no+[(2,trial,n)]
                trialno2=trialno2+[(2,n)]
            if test3!=0:
                no=no+[(3,trial,n)]
                trialno3=trialno3+[(3,n)]
            print((trial,n,test2,test3))
    finish=time.time()
    print("----------------------------------------------------------------------------------------------")
    print("trial:",trial,"minutes:",(finish-start)/60)
    print("zs2:");print(zs2);print();print("nzs2:");print(nzs2);print()
    print("zs3:");print(zs3);print();print("nzs3:");print(nzs3);print()
    print("trialno p=2:");print(trialno2);print()
    print("trialno p=3:");print(trialno3)
print("=========================================================================================")
print("no:");print();print(no)

(1, 1, 0, 0)
(1, 2, 0, 0)
(1, 3, 0, 0)
(1, 4, 0, 0)
(1, 5, 0, 0)
(1, 6, 0, 0)
(1, 7, 0, 0)
(1, 8, 0, 0)
(1, 9, 0, 0)
(1, 10, 0, 0)
(1, 11, 0, 0)
(1, 12, 0, 0)
(1, 13, 0, 0)
(1, 14, 0, 0)
(1, 15, 0, 0)
(1, 16, 0, 0)
(1, 17, 0, 0)
(1, 18, 0, 0)
(1, 19, 0, 0)
(1, 20, 0, 0)
(1, 21, 0, 0)
(1, 22, 0, 0)
(1, 23, 0, 0)
(1, 24, 0, 0)
(1, 25, 0, 0)
(1, 26, 0, 0)
(1, 27, 0, 0)
(1, 28, 0, 0)
(1, 29, 0, 0)
(1, 30, 0, 0)
(1, 31, 0, 0)
(1, 32, 0, 0)
(1, 33, 0, 0)
(1, 34, 0, 0)
(1, 35, 0, 0)
(1, 36, 0, 0)
(1, 37, 0, 0)
(1, 38, 0, 0)
(1, 39, 0, 0)
(1, 40, 0, 0)
(1, 41, 0, 0)
(1, 42, 0, 0)
(1, 43, 0, 0)
(1, 44, 0, 0)
(1, 45, 0, 0)
(1, 46, 0, 0)
(1, 47, 0, 0)
(1, 48, 0, 0)
(1, 49, 0, 0)
(1, 50, 0, 0)
(1, 51, 0, 0)
(1, 52, 0, 0)
(1, 53, 0, 0)
(1, 54, 0, 0)
(1, 55, 0, 0)
(1, 56, 0, 0)
(1, 57, 0, 0)
(1, 58, 0, 0)
(1, 59, 0, 0)
(1, 60, 0, 0)
(1, 61, 0, 0)
(1, 62, 0, 0)
(1, 63, 0, 0)
(1, 64, 0, 0)
(1, 65, 0, 0)
(1, 66, 0, 0)
(1, 67, 0, 0)
(1, 68, 0, 0)
(1, 69, 0, 0)
(1, 70, 0, 0)
(1, 71, 0, 0)
(1, 72, 0, 0)
(

(4, 55, 0, 0)
(4, 56, 0, 0)
(4, 57, 0, 0)
(4, 58, 0, 0)
(4, 59, 0, 0)
(4, 60, 0, 0)
(4, 61, 0, 0)
(4, 62, 0, 0)
(4, 63, 0, 0)
(4, 64, 0, 0)
(4, 65, 0, 0)
(4, 66, 0, 0)
(4, 67, 0, 0)
(4, 68, 0, 0)
(4, 69, 0, 0)
(4, 70, 0, 0)
(4, 71, 0, 0)
(4, 72, 0, 0)
(4, 73, 0, 0)
(4, 74, 0, 0)
(4, 75, 0, 0)
(4, 76, 0, 0)
(4, 77, 0, 0)
(4, 78, 0, 0)
(4, 79, 0, 0)
(4, 80, 0, 0)
(4, 81, 0, 0)
(4, 82, 0, 0)
(4, 83, 0, 0)
(4, 84, 0, 0)
(4, 85, 0, 0)
(4, 86, 0, 0)
(4, 87, 0, 0)
(4, 88, 0, 0)
(4, 89, 0, 0)
(4, 90, 0, 0)
(4, 91, 0, 0)
(4, 92, 0, 0)
(4, 93, 0, 0)
(4, 94, 0, 0)
(4, 95, 0, 0)
(4, 96, 0, 0)
(4, 97, 0, 0)
(4, 98, 0, 0)
(4, 99, 0, 0)
(4, 100, 0, 0)
----------------------------------------------------------------------------------------------
trial: 4 minutes: 0.6098387320836385
zs2:
[]

nzs2:
[1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 

(8, 2, 0, 0)
(8, 3, 0, 0)
(8, 4, 0, 0)
(8, 5, 0, 0)
(8, 6, 0, 0)
(8, 7, 0, 0)
(8, 8, 0, 0)
(8, 9, 0, 0)
(8, 10, 0, 0)
(8, 11, 0, 0)
(8, 12, 0, 0)
(8, 13, 0, 0)
(8, 14, 0, 0)
(8, 15, 0, 0)
(8, 16, 0, 0)
(8, 17, 0, 0)
(8, 18, 0, 0)
(8, 19, 0, 0)
(8, 20, 0, 0)
(8, 21, 0, 0)
(8, 22, 0, 0)
(8, 23, 0, 0)
(8, 24, 0, 0)
(8, 25, 0, 0)
(8, 26, 0, 0)
(8, 27, 0, 0)
(8, 28, 0, 0)
(8, 29, 0, 0)
(8, 30, 0, 0)
(8, 31, 0, 0)
(8, 32, 0, 0)
(8, 33, 0, 0)
(8, 34, 0, 0)
(8, 35, 0, 0)
(8, 36, 0, 0)
(8, 37, 0, 0)
(8, 38, 0, 0)
(8, 39, 0, 0)
(8, 40, 0, 0)
(8, 41, 0, 0)
(8, 42, 0, 0)
(8, 43, 0, 0)
(8, 44, 0, 0)
(8, 45, 0, 0)
(8, 46, 0, 0)
(8, 47, 0, 0)
(8, 48, 0, 0)
(8, 49, 0, 0)
(8, 50, 0, 0)
(8, 51, 0, 0)
(8, 52, 0, 0)
(8, 53, 0, 0)
(8, 54, 0, 0)
(8, 55, 0, 0)
(8, 56, 0, 0)
(8, 57, 0, 0)
(8, 58, 0, 0)
(8, 59, 0, 0)
(8, 60, 0, 0)
(8, 61, 0, 0)
(8, 62, 0, 0)
(8, 63, 0, 0)
(8, 64, 0, 0)
(8, 65, 0, 0)
(8, 66, 0, 0)
(8, 67, 0, 0)
(8, 68, 0, 0)
(8, 69, 0, 0)
(8, 70, 0, 0)
(8, 71, 0, 0)
(8, 72, 0, 0)
(8, 73, 0, 0)


In [3]:
L=LaurentSeriesRing(QQ,'x', default_prec=1050)
import random
import time
def specialModulus(a):
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    return 6*8^ds2*3^ds3

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    randoms=[]
    for k in [1..n+2]:
        randoms=randoms+[ZZ(random.randint(-50, 50))]
    for k in [1..n+1]:
        sm=specialModulus(n)# n, not k.
        term=randoms[k]*sm+rmnjntau(k) # k, not n.
        ans=ans+term*x^k
    return ans

def recip(n):
    return L(1/RamanujanCongruenceSeries(n)).O(n+1)

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]

    
no=[]
for trial in [1..20]:
    zs2=[]
    zs3=[]
    nzs2=[]
    nzs3=[]
    trialno2=[]
    trialno3=[]
    start=time.time()
    print()
    for n in [1..300]:
        c=recipPowerConstantTerm(n)
        if c==0:
            zs2=zs2+[mod(n,2)]
            zs3=zs3+[(mod(n,3))]
        if c!=0:
            nzs2=nzs2+[mod(n,2)]
            nzs3=nzs3+[mod(n,3)]
            o2=order(2,c)
            sd2=digitsum(2,n)
            nzs2=nzs2+[mod(n,2)]
            o3=order(3,c)
            sd3=digitsum(3,n)
            test2=3*sd2-o2
            test3=sd3-o3
            if test2!=0:
                no=no+[(2,trial,n)]
                trialno2=trialno2+[(2,n)]
            if test3!=0:
                no=no+[(3,trial,n)]
                trialno3=trialno3+[(3,n)]
            print((trial,n,test2,test3,len(no)))
    finish=time.time()
    print("----------------------------------------------------------------------------------------------")
    print("trial:",trial,"minutes:",(finish-start)/60)
    print("zs2:");print(zs2);print();print("nzs2:");print(nzs2);print()
    print("zs3:");print(zs3);print();print("nzs3:");print(nzs3);print()
    print("trialno p=2:");print(trialno2);print()
    print("trialno p=3:");print(trialno3)
print("=========================================================================================")
print("no:");print();print(no)


(1, 1, 0, 0, 0)
(1, 2, 0, 0, 0)
(1, 3, 0, 0, 0)
(1, 4, 0, 0, 0)
(1, 5, 0, 0, 0)
(1, 6, 0, 0, 0)
(1, 7, 0, 0, 0)
(1, 8, 0, 0, 0)
(1, 9, 0, 0, 0)
(1, 10, 0, 0, 0)
(1, 11, 0, 0, 0)
(1, 12, 0, 0, 0)
(1, 13, 0, 0, 0)
(1, 14, 0, 0, 0)
(1, 15, 0, 0, 0)
(1, 16, 0, 0, 0)
(1, 17, 0, 0, 0)
(1, 18, 0, 0, 0)
(1, 19, 0, 0, 0)
(1, 20, 0, 0, 0)
(1, 21, 0, 0, 0)
(1, 22, 0, 0, 0)
(1, 23, 0, 0, 0)
(1, 24, 0, 0, 0)
(1, 25, 0, 0, 0)
(1, 26, 0, 0, 0)
(1, 27, 0, 0, 0)
(1, 28, 0, 0, 0)
(1, 29, 0, 0, 0)
(1, 30, 0, 0, 0)
(1, 31, 0, 0, 0)
(1, 32, 0, 0, 0)
(1, 33, 0, 0, 0)
(1, 34, 0, 0, 0)
(1, 35, 0, 0, 0)
(1, 36, 0, 0, 0)
(1, 37, 0, 0, 0)
(1, 38, 0, 0, 0)
(1, 39, 0, 0, 0)
(1, 40, 0, 0, 0)
(1, 41, 0, 0, 0)
(1, 42, 0, 0, 0)
(1, 43, 0, 0, 0)
(1, 44, 0, 0, 0)
(1, 45, 0, 0, 0)
(1, 46, 0, 0, 0)
(1, 47, 0, 0, 0)
(1, 48, 0, 0, 0)
(1, 49, 0, 0, 0)
(1, 50, 0, 0, 0)
(1, 51, 0, 0, 0)
(1, 52, 0, 0, 0)
(1, 53, 0, 0, 0)
(1, 54, 0, 0, 0)
(1, 55, 0, 0, 0)
(1, 56, 0, 0, 0)
(1, 57, 0, 0, 0)
(1, 58, 0, 0, 0)
(1, 59, 0, 0, 0)
(1, 6

(2, 3, 0, 0, 0)
(2, 4, 0, 0, 0)
(2, 5, 0, 0, 0)
(2, 6, 0, 0, 0)
(2, 7, 0, 0, 0)
(2, 8, 0, 0, 0)
(2, 9, 0, 0, 0)
(2, 10, 0, 0, 0)
(2, 11, 0, 0, 0)
(2, 12, 0, 0, 0)
(2, 13, 0, 0, 0)
(2, 14, 0, 0, 0)
(2, 15, 0, 0, 0)
(2, 16, 0, 0, 0)
(2, 17, 0, 0, 0)
(2, 18, 0, 0, 0)
(2, 19, 0, 0, 0)
(2, 20, 0, 0, 0)
(2, 21, 0, 0, 0)
(2, 22, 0, 0, 0)
(2, 23, 0, 0, 0)
(2, 24, 0, 0, 0)
(2, 25, 0, 0, 0)
(2, 26, 0, 0, 0)
(2, 27, 0, 0, 0)
(2, 28, 0, 0, 0)
(2, 29, 0, 0, 0)
(2, 30, 0, 0, 0)
(2, 31, 0, 0, 0)
(2, 32, 0, 0, 0)
(2, 33, 0, 0, 0)
(2, 34, 0, 0, 0)
(2, 35, 0, 0, 0)
(2, 36, 0, 0, 0)
(2, 37, 0, 0, 0)
(2, 38, 0, 0, 0)
(2, 39, 0, 0, 0)
(2, 40, 0, 0, 0)
(2, 41, 0, 0, 0)
(2, 42, 0, 0, 0)
(2, 43, 0, 0, 0)
(2, 44, 0, 0, 0)
(2, 45, 0, 0, 0)
(2, 46, 0, 0, 0)
(2, 47, 0, 0, 0)
(2, 48, 0, 0, 0)
(2, 49, 0, 0, 0)
(2, 50, 0, 0, 0)
(2, 51, 0, 0, 0)
(2, 52, 0, 0, 0)
(2, 53, 0, 0, 0)
(2, 54, 0, 0, 0)
(2, 55, 0, 0, 0)
(2, 56, 0, 0, 0)
(2, 57, 0, 0, 0)
(2, 58, 0, 0, 0)
(2, 59, 0, 0, 0)
(2, 60, 0, 0, 0)
(2, 61, 0, 0, 0)
(2, 

(3, 6, 0, 0, 0)
(3, 7, 0, 0, 0)
(3, 8, 0, 0, 0)
(3, 9, 0, 0, 0)
(3, 10, 0, 0, 0)
(3, 11, 0, 0, 0)
(3, 12, 0, 0, 0)
(3, 13, 0, 0, 0)
(3, 14, 0, 0, 0)
(3, 15, 0, 0, 0)
(3, 16, 0, 0, 0)
(3, 17, 0, 0, 0)
(3, 18, 0, 0, 0)
(3, 19, 0, 0, 0)
(3, 20, 0, 0, 0)
(3, 21, 0, 0, 0)
(3, 22, 0, 0, 0)
(3, 23, 0, 0, 0)
(3, 24, 0, 0, 0)
(3, 25, 0, 0, 0)
(3, 26, 0, 0, 0)
(3, 27, 0, 0, 0)
(3, 28, 0, 0, 0)
(3, 29, 0, 0, 0)
(3, 30, 0, 0, 0)
(3, 31, 0, 0, 0)
(3, 32, 0, 0, 0)
(3, 33, 0, 0, 0)
(3, 34, 0, 0, 0)
(3, 35, 0, 0, 0)
(3, 36, 0, 0, 0)
(3, 37, 0, 0, 0)
(3, 38, 0, 0, 0)
(3, 39, 0, 0, 0)
(3, 40, 0, 0, 0)
(3, 41, 0, 0, 0)
(3, 42, 0, 0, 0)
(3, 43, 0, 0, 0)
(3, 44, 0, 0, 0)
(3, 45, 0, 0, 0)
(3, 46, 0, 0, 0)
(3, 47, 0, 0, 0)
(3, 48, 0, 0, 0)
(3, 49, 0, 0, 0)
(3, 50, 0, 0, 0)
(3, 51, 0, 0, 0)
(3, 52, 0, 0, 0)
(3, 53, 0, 0, 0)
(3, 54, 0, 0, 0)
(3, 55, 0, 0, 0)
(3, 56, 0, 0, 0)
(3, 57, 0, 0, 0)
(3, 58, 0, 0, 0)
(3, 59, 0, 0, 0)
(3, 60, 0, 0, 0)
(3, 61, 0, 0, 0)
(3, 62, 0, 0, 0)
(3, 63, 0, 0, 0)
(3, 64, 0, 0, 0)
(

(4, 7, 0, 0, 0)
(4, 8, 0, 0, 0)
(4, 9, 0, 0, 0)
(4, 10, 0, 0, 0)
(4, 11, 0, 0, 0)
(4, 12, 0, 0, 0)
(4, 13, 0, 0, 0)
(4, 14, 0, 0, 0)
(4, 15, 0, 0, 0)
(4, 16, 0, 0, 0)
(4, 17, 0, 0, 0)
(4, 18, 0, 0, 0)
(4, 19, 0, 0, 0)
(4, 20, 0, 0, 0)
(4, 21, 0, 0, 0)
(4, 22, 0, 0, 0)
(4, 23, 0, 0, 0)
(4, 24, 0, 0, 0)
(4, 25, 0, 0, 0)
(4, 26, 0, 0, 0)
(4, 27, 0, 0, 0)
(4, 28, 0, 0, 0)
(4, 29, 0, 0, 0)
(4, 30, 0, 0, 0)
(4, 31, 0, 0, 0)
(4, 32, 0, 0, 0)
(4, 33, 0, 0, 0)
(4, 34, 0, 0, 0)
(4, 35, 0, 0, 0)
(4, 36, 0, 0, 0)
(4, 37, 0, 0, 0)
(4, 38, 0, 0, 0)
(4, 39, 0, 0, 0)
(4, 40, 0, 0, 0)
(4, 41, 0, 0, 0)
(4, 42, 0, 0, 0)
(4, 43, 0, 0, 0)
(4, 44, 0, 0, 0)
(4, 45, 0, 0, 0)
(4, 46, 0, 0, 0)
(4, 47, 0, 0, 0)
(4, 48, 0, 0, 0)
(4, 49, 0, 0, 0)
(4, 50, 0, 0, 0)
(4, 51, 0, 0, 0)
(4, 52, 0, 0, 0)
(4, 53, 0, 0, 0)
(4, 54, 0, 0, 0)
(4, 55, 0, 0, 0)
(4, 56, 0, 0, 0)
(4, 57, 0, 0, 0)
(4, 58, 0, 0, 0)
(4, 59, 0, 0, 0)
(4, 60, 0, 0, 0)
(4, 61, 0, 0, 0)
(4, 62, 0, 0, 0)
(4, 63, 0, 0, 0)
(4, 64, 0, 0, 0)
(4, 65, 0, 0, 0)


(5, 10, 0, 0, 0)
(5, 11, 0, 0, 0)
(5, 12, 0, 0, 0)
(5, 13, 0, 0, 0)
(5, 14, 0, 0, 0)
(5, 15, 0, 0, 0)
(5, 16, 0, 0, 0)
(5, 17, 0, 0, 0)
(5, 18, 0, 0, 0)
(5, 19, 0, 0, 0)
(5, 20, 0, 0, 0)
(5, 21, 0, 0, 0)
(5, 22, 0, 0, 0)
(5, 23, 0, 0, 0)
(5, 24, 0, 0, 0)
(5, 25, 0, 0, 0)
(5, 26, 0, 0, 0)
(5, 27, 0, 0, 0)
(5, 28, 0, 0, 0)
(5, 29, 0, 0, 0)
(5, 30, 0, 0, 0)
(5, 31, 0, 0, 0)
(5, 32, 0, 0, 0)
(5, 33, 0, 0, 0)
(5, 34, 0, 0, 0)
(5, 35, 0, 0, 0)
(5, 36, 0, 0, 0)
(5, 37, 0, 0, 0)
(5, 38, 0, 0, 0)
(5, 39, 0, 0, 0)
(5, 40, 0, 0, 0)
(5, 41, 0, 0, 0)
(5, 42, 0, 0, 0)
(5, 43, 0, 0, 0)
(5, 44, 0, 0, 0)
(5, 45, 0, 0, 0)
(5, 46, 0, 0, 0)
(5, 47, 0, 0, 0)
(5, 48, 0, 0, 0)
(5, 49, 0, 0, 0)
(5, 50, 0, 0, 0)
(5, 51, 0, 0, 0)
(5, 52, 0, 0, 0)
(5, 53, 0, 0, 0)
(5, 54, 0, 0, 0)
(5, 55, 0, 0, 0)
(5, 56, 0, 0, 0)
(5, 57, 0, 0, 0)
(5, 58, 0, 0, 0)
(5, 59, 0, 0, 0)
(5, 60, 0, 0, 0)
(5, 61, 0, 0, 0)
(5, 62, 0, 0, 0)
(5, 63, 0, 0, 0)
(5, 64, 0, 0, 0)
(5, 65, 0, 0, 0)
(5, 66, 0, 0, 0)
(5, 67, 0, 0, 0)
(5, 68, 0, 0, 

...

(16, 3, 0, 0, 0)
(16, 4, 0, 0, 0)
(16, 5, 0, 0, 0)
(16, 6, 0, 0, 0)
(16, 7, 0, 0, 0)
(16, 8, 0, 0, 0)
(16, 9, 0, 0, 0)
(16, 10, 0, 0, 0)
(16, 11, 0, 0, 0)
(16, 12, 0, 0, 0)
(16, 13, 0, 0, 0)
(16, 14, 0, 0, 0)
(16, 15, 0, 0, 0)
(16, 16, 0, 0, 0)
(16, 17, 0, 0, 0)
(16, 18, 0, 0, 0)
(16, 19, 0, 0, 0)
(16, 20, 0, 0, 0)
(16, 21, 0, 0, 0)
(16, 22, 0, 0, 0)
(16, 23, 0, 0, 0)
(16, 24, 0, 0, 0)
(16, 25, 0, 0, 0)
(16, 26, 0, 0, 0)
(16, 27, 0, 0, 0)
(16, 28, 0, 0, 0)
(16, 29, 0, 0, 0)
(16, 30, 0, 0, 0)
(16, 31, 0, 0, 0)
(16, 32, 0, 0, 0)
(16, 33, 0, 0, 0)
(16, 34, 0, 0, 0)
(16, 35, 0, 0, 0)
(16, 36, 0, 0, 0)
(16, 37, 0, 0, 0)
(16, 38, 0, 0, 0)
(16, 39, 0, 0, 0)
(16, 40, 0, 0, 0)
(16, 41, 0, 0, 0)
(16, 42, 0, 0, 0)
(16, 43, 0, 0, 0)
(16, 44, 0, 0, 0)
(16, 45, 0, 0, 0)
(16, 46, 0, 0, 0)
(16, 47, 0, 0, 0)
(16, 48, 0, 0, 0)
(16, 49, 0, 0, 0)
(16, 50, 0, 0, 0)
(16, 51, 0, 0, 0)
(16, 52, 0, 0, 0)
(16, 53, 0, 0, 0)
(16, 54, 0, 0, 0)
(16, 55, 0, 0, 0)
(16, 56, 0, 0, 0)
(16, 57, 0, 0, 0)
(16, 58, 0, 0, 0)

(17, 2, 0, 0, 0)
(17, 3, 0, 0, 0)
(17, 4, 0, 0, 0)
(17, 5, 0, 0, 0)
(17, 6, 0, 0, 0)
(17, 7, 0, 0, 0)
(17, 8, 0, 0, 0)
(17, 9, 0, 0, 0)
(17, 10, 0, 0, 0)
(17, 11, 0, 0, 0)
(17, 12, 0, 0, 0)
(17, 13, 0, 0, 0)
(17, 14, 0, 0, 0)
(17, 15, 0, 0, 0)
(17, 16, 0, 0, 0)
(17, 17, 0, 0, 0)
(17, 18, 0, 0, 0)
(17, 19, 0, 0, 0)
(17, 20, 0, 0, 0)
(17, 21, 0, 0, 0)
(17, 22, 0, 0, 0)
(17, 23, 0, 0, 0)
(17, 24, 0, 0, 0)
(17, 25, 0, 0, 0)
(17, 26, 0, 0, 0)
(17, 27, 0, 0, 0)
(17, 28, 0, 0, 0)
(17, 29, 0, 0, 0)
(17, 30, 0, 0, 0)
(17, 31, 0, 0, 0)
(17, 32, 0, 0, 0)
(17, 33, 0, 0, 0)
(17, 34, 0, 0, 0)
(17, 35, 0, 0, 0)
(17, 36, 0, 0, 0)
(17, 37, 0, 0, 0)
(17, 38, 0, 0, 0)
(17, 39, 0, 0, 0)
(17, 40, 0, 0, 0)
(17, 41, 0, 0, 0)
(17, 42, 0, 0, 0)
(17, 43, 0, 0, 0)
(17, 44, 0, 0, 0)
(17, 45, 0, 0, 0)
(17, 46, 0, 0, 0)
(17, 47, 0, 0, 0)
(17, 48, 0, 0, 0)
(17, 49, 0, 0, 0)
(17, 50, 0, 0, 0)
(17, 51, 0, 0, 0)
(17, 52, 0, 0, 0)
(17, 53, 0, 0, 0)
(17, 54, 0, 0, 0)
(17, 55, 0, 0, 0)
(17, 56, 0, 0, 0)
(17, 57, 0, 0, 0)


(18, 2, 0, 0, 0)
(18, 3, 0, 0, 0)
(18, 4, 0, 0, 0)
(18, 5, 0, 0, 0)
(18, 6, 0, 0, 0)
(18, 7, 0, 0, 0)
(18, 8, 0, 0, 0)
(18, 9, 0, 0, 0)
(18, 10, 0, 0, 0)
(18, 11, 0, 0, 0)
(18, 12, 0, 0, 0)
(18, 13, 0, 0, 0)
(18, 14, 0, 0, 0)
(18, 15, 0, 0, 0)
(18, 16, 0, 0, 0)
(18, 17, 0, 0, 0)
(18, 18, 0, 0, 0)
(18, 19, 0, 0, 0)
(18, 20, 0, 0, 0)
(18, 21, 0, 0, 0)
(18, 22, 0, 0, 0)
(18, 23, 0, 0, 0)
(18, 24, 0, 0, 0)
(18, 25, 0, 0, 0)
(18, 26, 0, 0, 0)
(18, 27, 0, 0, 0)
(18, 28, 0, 0, 0)
(18, 29, 0, 0, 0)
(18, 30, 0, 0, 0)
(18, 31, 0, 0, 0)
(18, 32, 0, 0, 0)
(18, 33, 0, 0, 0)
(18, 34, 0, 0, 0)
(18, 35, 0, 0, 0)
(18, 36, 0, 0, 0)
(18, 37, 0, 0, 0)
(18, 38, 0, 0, 0)
(18, 39, 0, 0, 0)
(18, 40, 0, 0, 0)
(18, 41, 0, 0, 0)
(18, 42, 0, 0, 0)
(18, 43, 0, 0, 0)
(18, 44, 0, 0, 0)
(18, 45, 0, 0, 0)
(18, 46, 0, 0, 0)
(18, 47, 0, 0, 0)
(18, 48, 0, 0, 0)
(18, 49, 0, 0, 0)
(18, 50, 0, 0, 0)
(18, 51, 0, 0, 0)
(18, 52, 0, 0, 0)
(18, 53, 0, 0, 0)
(18, 54, 0, 0, 0)
(18, 55, 0, 0, 0)
(18, 56, 0, 0, 0)
(18, 57, 0, 0, 0)


(19, 2, 0, 0, 0)
(19, 3, 0, 0, 0)
(19, 4, 0, 0, 0)
(19, 5, 0, 0, 0)
(19, 6, 0, 0, 0)
(19, 7, 0, 0, 0)
(19, 8, 0, 0, 0)
(19, 9, 0, 0, 0)
(19, 10, 0, 0, 0)
(19, 11, 0, 0, 0)
(19, 12, 0, 0, 0)
(19, 13, 0, 0, 0)
(19, 14, 0, 0, 0)
(19, 15, 0, 0, 0)
(19, 16, 0, 0, 0)
(19, 17, 0, 0, 0)
(19, 18, 0, 0, 0)
(19, 19, 0, 0, 0)
(19, 20, 0, 0, 0)
(19, 21, 0, 0, 0)
(19, 22, 0, 0, 0)
(19, 23, 0, 0, 0)
(19, 24, 0, 0, 0)
(19, 25, 0, 0, 0)
(19, 26, 0, 0, 0)
(19, 27, 0, 0, 0)
(19, 28, 0, 0, 0)
(19, 29, 0, 0, 0)
(19, 30, 0, 0, 0)
(19, 31, 0, 0, 0)
(19, 32, 0, 0, 0)
(19, 33, 0, 0, 0)
(19, 34, 0, 0, 0)
(19, 35, 0, 0, 0)
(19, 36, 0, 0, 0)
(19, 37, 0, 0, 0)
(19, 38, 0, 0, 0)
(19, 39, 0, 0, 0)
(19, 40, 0, 0, 0)
(19, 41, 0, 0, 0)
(19, 42, 0, 0, 0)
(19, 43, 0, 0, 0)
(19, 44, 0, 0, 0)
(19, 45, 0, 0, 0)
(19, 46, 0, 0, 0)
(19, 47, 0, 0, 0)
(19, 48, 0, 0, 0)
(19, 49, 0, 0, 0)
(19, 50, 0, 0, 0)
(19, 51, 0, 0, 0)
(19, 52, 0, 0, 0)
(19, 53, 0, 0, 0)
(19, 54, 0, 0, 0)
(19, 55, 0, 0, 0)
(19, 56, 0, 0, 0)
(19, 57, 0, 0, 0)


(20, 2, 0, 0, 0)
(20, 3, 0, 0, 0)
(20, 4, 0, 0, 0)
(20, 5, 0, 0, 0)
(20, 6, 0, 0, 0)
(20, 7, 0, 0, 0)
(20, 8, 0, 0, 0)
(20, 9, 0, 0, 0)
(20, 10, 0, 0, 0)
(20, 11, 0, 0, 0)
(20, 12, 0, 0, 0)
(20, 13, 0, 0, 0)
(20, 14, 0, 0, 0)
(20, 15, 0, 0, 0)
(20, 16, 0, 0, 0)
(20, 17, 0, 0, 0)
(20, 18, 0, 0, 0)
(20, 19, 0, 0, 0)
(20, 20, 0, 0, 0)
(20, 21, 0, 0, 0)
(20, 22, 0, 0, 0)
(20, 23, 0, 0, 0)
(20, 24, 0, 0, 0)
(20, 25, 0, 0, 0)
(20, 26, 0, 0, 0)
(20, 27, 0, 0, 0)
(20, 28, 0, 0, 0)
(20, 29, 0, 0, 0)
(20, 30, 0, 0, 0)
(20, 31, 0, 0, 0)
(20, 32, 0, 0, 0)
(20, 33, 0, 0, 0)
(20, 34, 0, 0, 0)
(20, 35, 0, 0, 0)
(20, 36, 0, 0, 0)
(20, 37, 0, 0, 0)
(20, 38, 0, 0, 0)
(20, 39, 0, 0, 0)
(20, 40, 0, 0, 0)
(20, 41, 0, 0, 0)
(20, 42, 0, 0, 0)
(20, 43, 0, 0, 0)
(20, 44, 0, 0, 0)
(20, 45, 0, 0, 0)
(20, 46, 0, 0, 0)
(20, 47, 0, 0, 0)
(20, 48, 0, 0, 0)
(20, 49, 0, 0, 0)
(20, 50, 0, 0, 0)
(20, 51, 0, 0, 0)
(20, 52, 0, 0, 0)
(20, 53, 0, 0, 0)
(20, 54, 0, 0, 0)
(20, 55, 0, 0, 0)
(20, 56, 0, 0, 0)
(20, 57, 0, 0, 0)


In [4]:
L=LaurentSeriesRing(QQ,'x', default_prec=1050)
import random
import time
def specialModulus(a):
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    return 6*8^ds2*3^ds3

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    randoms=[]
    for k in [1..n+2]:
        randoms=randoms+[ZZ(random.randint(-50, 50))]
    for k in [1..n+1]:
        sm=specialModulus(n)# n, not k.
        term=randoms[k]*sm+rmnjntau(k) # k, not n.
        ans=ans+term*x^k
    return ans

def recip(n):
    return L(1/RamanujanCongruenceSeries(n)).O(n+1)

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]

    
no=[]
for trial in [1..5]:
    zs2=[]
    zs3=[]
    nzs2=[]
    nzs3=[]
    trialno2=[]
    trialno3=[]
    start=time.time()
    print()
    for n in [1..1000]:
        c=recipPowerConstantTerm(n)
        if c==0:
            zs2=zs2+[mod(n,2)]
            zs3=zs3+[(mod(n,3))]
        if c!=0:
            nzs2=nzs2+[mod(n,2)]
            nzs3=nzs3+[mod(n,3)]
            o2=order(2,c)
            sd2=digitsum(2,n)
            nzs2=nzs2+[mod(n,2)]
            o3=order(3,c)
            sd3=digitsum(3,n)
            test2=3*sd2-o2
            test3=sd3-o3
            if test2!=0:
                no=no+[(2,trial,n)]
                trialno2=trialno2+[(2,n)]
            if test3!=0:
                no=no+[(3,trial,n)]
                trialno3=trialno3+[(3,n)]
            print((trial,n,test2,test3,len(no)))
    finish=time.time()
    print("----------------------------------------------------------------------------------------------")
    print("trial:",trial,"minutes:",(finish-start)/60)
    print("zs2:");print(zs2);print();print("nzs2:");print(nzs2);print()
    print("zs3:");print(zs3);print();print("nzs3:");print(nzs3);print()
    print("trialno p=2:");print(trialno2);print()
    print("trialno p=3:");print(trialno3)
print("=========================================================================================")
print("no:");print();print(no)


(1, 1, 0, 0, 0)
(1, 2, 0, 0, 0)
(1, 3, 0, 0, 0)
(1, 4, 0, 0, 0)
(1, 5, 0, 0, 0)
(1, 6, 0, 0, 0)
(1, 7, 0, 0, 0)
(1, 8, 0, 0, 0)
(1, 9, 0, 0, 0)
(1, 10, 0, 0, 0)
(1, 11, 0, 0, 0)
(1, 12, 0, 0, 0)
(1, 13, 0, 0, 0)
(1, 14, 0, 0, 0)
(1, 15, 0, 0, 0)
(1, 16, 0, 0, 0)
(1, 17, 0, 0, 0)
(1, 18, 0, 0, 0)
(1, 19, 0, 0, 0)
(1, 20, 0, 0, 0)
(1, 21, 0, 0, 0)
(1, 22, 0, 0, 0)
(1, 23, 0, 0, 0)
(1, 24, 0, 0, 0)
(1, 25, 0, 0, 0)
(1, 26, 0, 0, 0)
(1, 27, 0, 0, 0)
(1, 28, 0, 0, 0)
(1, 29, 0, 0, 0)
(1, 30, 0, 0, 0)
(1, 31, 0, 0, 0)
(1, 32, 0, 0, 0)
(1, 33, 0, 0, 0)
(1, 34, 0, 0, 0)
(1, 35, 0, 0, 0)
(1, 36, 0, 0, 0)
(1, 37, 0, 0, 0)
(1, 38, 0, 0, 0)
(1, 39, 0, 0, 0)
(1, 40, 0, 0, 0)
(1, 41, 0, 0, 0)
(1, 42, 0, 0, 0)
(1, 43, 0, 0, 0)
(1, 44, 0, 0, 0)
(1, 45, 0, 0, 0)
(1, 46, 0, 0, 0)
(1, 47, 0, 0, 0)
(1, 48, 0, 0, 0)
(1, 49, 0, 0, 0)
(1, 50, 0, 0, 0)
(1, 51, 0, 0, 0)
(1, 52, 0, 0, 0)
(1, 53, 0, 0, 0)
(1, 54, 0, 0, 0)
(1, 55, 0, 0, 0)
(1, 56, 0, 0, 0)
(1, 57, 0, 0, 0)
(1, 58, 0, 0, 0)
(1, 59, 0, 0, 0)
(1, 6

(1, 463, 0, 0, 0)
(1, 464, 0, 0, 0)
(1, 465, 0, 0, 0)
(1, 466, 0, 0, 0)
(1, 467, 0, 0, 0)
(1, 468, 0, 0, 0)
(1, 469, 0, 0, 0)
(1, 470, 0, 0, 0)
(1, 471, 0, 0, 0)
(1, 472, 0, 0, 0)
(1, 473, 0, 0, 0)
(1, 474, 0, 0, 0)
(1, 475, 0, 0, 0)
(1, 476, 0, 0, 0)
(1, 477, 0, 0, 0)
(1, 478, 0, 0, 0)
(1, 479, 0, 0, 0)
(1, 480, 0, 0, 0)
(1, 481, 0, 0, 0)
(1, 482, 0, 0, 0)
(1, 483, 0, 0, 0)
(1, 484, 0, 0, 0)
(1, 485, 0, 0, 0)
(1, 486, 0, 0, 0)
(1, 487, 0, 0, 0)
(1, 488, 0, 0, 0)
(1, 489, 0, 0, 0)
(1, 490, 0, 0, 0)
(1, 491, 0, 0, 0)
(1, 492, 0, 0, 0)
(1, 493, 0, 0, 0)
(1, 494, 0, 0, 0)
(1, 495, 0, 0, 0)
(1, 496, 0, 0, 0)
(1, 497, 0, 0, 0)
(1, 498, 0, 0, 0)
(1, 499, 0, 0, 0)
(1, 500, 0, 0, 0)
(1, 501, 0, 0, 0)
(1, 502, 0, 0, 0)
(1, 503, 0, 0, 0)
(1, 504, 0, 0, 0)
(1, 505, 0, 0, 0)
(1, 506, 0, 0, 0)
(1, 507, 0, 0, 0)
(1, 508, 0, 0, 0)
(1, 509, 0, 0, 0)
(1, 510, 0, 0, 0)
(1, 511, 0, 0, 0)
(1, 512, 0, 0, 0)
(1, 513, 0, 0, 0)
(1, 514, 0, 0, 0)
(1, 515, 0, 0, 0)
(1, 516, 0, 0, 0)
(1, 517, 0, 0, 0)
(1, 518, 0

(1, 919, 0, 0, 0)
(1, 920, 0, 0, 0)
(1, 921, 0, 0, 0)
(1, 922, 0, 0, 0)
(1, 923, 0, 0, 0)
(1, 924, 0, 0, 0)
(1, 925, 0, 0, 0)
(1, 926, 0, 0, 0)
(1, 927, 0, 0, 0)
(1, 928, 0, 0, 0)
(1, 929, 0, 0, 0)
(1, 930, 0, 0, 0)
(1, 931, 0, 0, 0)
(1, 932, 0, 0, 0)
(1, 933, 0, 0, 0)
(1, 934, 0, 0, 0)
(1, 935, 0, 0, 0)
(1, 936, 0, 0, 0)
(1, 937, 0, 0, 0)
(1, 938, 0, 0, 0)
(1, 939, 0, 0, 0)
(1, 940, 0, 0, 0)
(1, 941, 0, 0, 0)
(1, 942, 0, 0, 0)
(1, 943, 0, 0, 0)
(1, 944, 0, 0, 0)
(1, 945, 0, 0, 0)
(1, 946, 0, 0, 0)
(1, 947, 0, 0, 0)
(1, 948, 0, 0, 0)
(1, 949, 0, 0, 0)
(1, 950, 0, 0, 0)
(1, 951, 0, 0, 0)
(1, 952, 0, 0, 0)
(1, 953, 0, 0, 0)
(1, 954, 0, 0, 0)
(1, 955, 0, 0, 0)
(1, 956, 0, 0, 0)
(1, 957, 0, 0, 0)
(1, 958, 0, 0, 0)
(1, 959, 0, 0, 0)
(1, 960, 0, 0, 0)
(1, 961, 0, 0, 0)
(1, 962, 0, 0, 0)
(1, 963, 0, 0, 0)
(1, 964, 0, 0, 0)
(1, 965, 0, 0, 0)
(1, 966, 0, 0, 0)
(1, 967, 0, 0, 0)
(1, 968, 0, 0, 0)
(1, 969, 0, 0, 0)
(1, 970, 0, 0, 0)
(1, 971, 0, 0, 0)
(1, 972, 0, 0, 0)
(1, 973, 0, 0, 0)
(1, 974, 0

(2, 3, 0, 0, 0)
(2, 4, 0, 0, 0)
(2, 5, 0, 0, 0)
(2, 6, 0, 0, 0)
(2, 7, 0, 0, 0)
(2, 8, 0, 0, 0)
(2, 9, 0, 0, 0)
(2, 10, 0, 0, 0)
(2, 11, 0, 0, 0)
(2, 12, 0, 0, 0)
(2, 13, 0, 0, 0)
(2, 14, 0, 0, 0)
(2, 15, 0, 0, 0)
(2, 16, 0, 0, 0)
(2, 17, 0, 0, 0)
(2, 18, 0, 0, 0)
(2, 19, 0, 0, 0)
(2, 20, 0, 0, 0)
(2, 21, 0, 0, 0)
(2, 22, 0, 0, 0)
(2, 23, 0, 0, 0)
(2, 24, 0, 0, 0)
(2, 25, 0, 0, 0)
(2, 26, 0, 0, 0)
(2, 27, 0, 0, 0)
(2, 28, 0, 0, 0)
(2, 29, 0, 0, 0)
(2, 30, 0, 0, 0)
(2, 31, 0, 0, 0)
(2, 32, 0, 0, 0)
(2, 33, 0, 0, 0)
(2, 34, 0, 0, 0)
(2, 35, 0, 0, 0)
(2, 36, 0, 0, 0)
(2, 37, 0, 0, 0)
(2, 38, 0, 0, 0)
(2, 39, 0, 0, 0)
(2, 40, 0, 0, 0)
(2, 41, 0, 0, 0)
(2, 42, 0, 0, 0)
(2, 43, 0, 0, 0)
(2, 44, 0, 0, 0)
(2, 45, 0, 0, 0)
(2, 46, 0, 0, 0)
(2, 47, 0, 0, 0)
(2, 48, 0, 0, 0)
(2, 49, 0, 0, 0)
(2, 50, 0, 0, 0)
(2, 51, 0, 0, 0)
(2, 52, 0, 0, 0)
(2, 53, 0, 0, 0)
(2, 54, 0, 0, 0)
(2, 55, 0, 0, 0)
(2, 56, 0, 0, 0)
(2, 57, 0, 0, 0)
(2, 58, 0, 0, 0)
(2, 59, 0, 0, 0)
(2, 60, 0, 0, 0)
(2, 61, 0, 0, 0)
(2, 

(2, 464, 0, 0, 0)
(2, 465, 0, 0, 0)
(2, 466, 0, 0, 0)
(2, 467, 0, 0, 0)
(2, 468, 0, 0, 0)
(2, 469, 0, 0, 0)
(2, 470, 0, 0, 0)
(2, 471, 0, 0, 0)
(2, 472, 0, 0, 0)
(2, 473, 0, 0, 0)
(2, 474, 0, 0, 0)
(2, 475, 0, 0, 0)
(2, 476, 0, 0, 0)
(2, 477, 0, 0, 0)
(2, 478, 0, 0, 0)
(2, 479, 0, 0, 0)
(2, 480, 0, 0, 0)
(2, 481, 0, 0, 0)
(2, 482, 0, 0, 0)
(2, 483, 0, 0, 0)
(2, 484, 0, 0, 0)
(2, 485, 0, 0, 0)
(2, 486, 0, 0, 0)
(2, 487, 0, 0, 0)
(2, 488, 0, 0, 0)
(2, 489, 0, 0, 0)
(2, 490, 0, 0, 0)
(2, 491, 0, 0, 0)
(2, 492, 0, 0, 0)
(2, 493, 0, 0, 0)
(2, 494, 0, 0, 0)
(2, 495, 0, 0, 0)
(2, 496, 0, 0, 0)
(2, 497, 0, 0, 0)
(2, 498, 0, 0, 0)
(2, 499, 0, 0, 0)
(2, 500, 0, 0, 0)
(2, 501, 0, 0, 0)
(2, 502, 0, 0, 0)
(2, 503, 0, 0, 0)
(2, 504, 0, 0, 0)
(2, 505, 0, 0, 0)
(2, 506, 0, 0, 0)
(2, 507, 0, 0, 0)
(2, 508, 0, 0, 0)
(2, 509, 0, 0, 0)
(2, 510, 0, 0, 0)
(2, 511, 0, 0, 0)
(2, 512, 0, 0, 0)
(2, 513, 0, 0, 0)
(2, 514, 0, 0, 0)
(2, 515, 0, 0, 0)
(2, 516, 0, 0, 0)
(2, 517, 0, 0, 0)
(2, 518, 0, 0, 0)
(2, 519, 0

...

(4, 464, 0, 0, 0)
(4, 465, 0, 0, 0)
(4, 466, 0, 0, 0)
(4, 467, 0, 0, 0)
(4, 468, 0, 0, 0)
(4, 469, 0, 0, 0)
(4, 470, 0, 0, 0)
(4, 471, 0, 0, 0)
(4, 472, 0, 0, 0)
(4, 473, 0, 0, 0)
(4, 474, 0, 0, 0)
(4, 475, 0, 0, 0)
(4, 476, 0, 0, 0)
(4, 477, 0, 0, 0)
(4, 478, 0, 0, 0)
(4, 479, 0, 0, 0)
(4, 480, 0, 0, 0)
(4, 481, 0, 0, 0)
(4, 482, 0, 0, 0)
(4, 483, 0, 0, 0)
(4, 484, 0, 0, 0)
(4, 485, 0, 0, 0)
(4, 486, 0, 0, 0)
(4, 487, 0, 0, 0)
(4, 488, 0, 0, 0)
(4, 489, 0, 0, 0)
(4, 490, 0, 0, 0)
(4, 491, 0, 0, 0)
(4, 492, 0, 0, 0)
(4, 493, 0, 0, 0)
(4, 494, 0, 0, 0)
(4, 495, 0, 0, 0)
(4, 496, 0, 0, 0)
(4, 497, 0, 0, 0)
(4, 498, 0, 0, 0)
(4, 499, 0, 0, 0)
(4, 500, 0, 0, 0)
(4, 501, 0, 0, 0)
(4, 502, 0, 0, 0)
(4, 503, 0, 0, 0)
(4, 504, 0, 0, 0)
(4, 505, 0, 0, 0)
(4, 506, 0, 0, 0)
(4, 507, 0, 0, 0)
(4, 508, 0, 0, 0)
(4, 509, 0, 0, 0)
(4, 510, 0, 0, 0)
(4, 511, 0, 0, 0)
(4, 512, 0, 0, 0)
(4, 513, 0, 0, 0)
(4, 514, 0, 0, 0)
(4, 515, 0, 0, 0)
(4, 516, 0, 0, 0)
(4, 517, 0, 0, 0)
(4, 518, 0, 0, 0)
(4, 519, 0

(4, 920, 0, 0, 0)
(4, 921, 0, 0, 0)
(4, 922, 0, 0, 0)
(4, 923, 0, 0, 0)
(4, 924, 0, 0, 0)
(4, 925, 0, 0, 0)
(4, 926, 0, 0, 0)
(4, 927, 0, 0, 0)
(4, 928, 0, 0, 0)
(4, 929, 0, 0, 0)
(4, 930, 0, 0, 0)
(4, 931, 0, 0, 0)
(4, 932, 0, 0, 0)
(4, 933, 0, 0, 0)
(4, 934, 0, 0, 0)
(4, 935, 0, 0, 0)
(4, 936, 0, 0, 0)
(4, 937, 0, 0, 0)
(4, 938, 0, 0, 0)
(4, 939, 0, 0, 0)
(4, 940, 0, 0, 0)
(4, 941, 0, 0, 0)
(4, 942, 0, 0, 0)
(4, 943, 0, 0, 0)
(4, 944, 0, 0, 0)
(4, 945, 0, 0, 0)
(4, 946, 0, 0, 0)
(4, 947, 0, 0, 0)
(4, 948, 0, 0, 0)
(4, 949, 0, 0, 0)
(4, 950, 0, 0, 0)
(4, 951, 0, 0, 0)
(4, 952, 0, 0, 0)
(4, 953, 0, 0, 0)
(4, 954, 0, 0, 0)
(4, 955, 0, 0, 0)
(4, 956, 0, 0, 0)
(4, 957, 0, 0, 0)
(4, 958, 0, 0, 0)
(4, 959, 0, 0, 0)
(4, 960, 0, 0, 0)
(4, 961, 0, 0, 0)
(4, 962, 0, 0, 0)
(4, 963, 0, 0, 0)
(4, 964, 0, 0, 0)
(4, 965, 0, 0, 0)
(4, 966, 0, 0, 0)
(4, 967, 0, 0, 0)
(4, 968, 0, 0, 0)
(4, 969, 0, 0, 0)
(4, 970, 0, 0, 0)
(4, 971, 0, 0, 0)
(4, 972, 0, 0, 0)
(4, 973, 0, 0, 0)
(4, 974, 0, 0, 0)
(4, 975, 0

(5, 3, 0, 0, 0)
(5, 4, 0, 0, 0)
(5, 5, 0, 0, 0)
(5, 6, 0, 0, 0)
(5, 7, 0, 0, 0)
(5, 8, 0, 0, 0)
(5, 9, 0, 0, 0)
(5, 10, 0, 0, 0)
(5, 11, 0, 0, 0)
(5, 12, 0, 0, 0)
(5, 13, 0, 0, 0)
(5, 14, 0, 0, 0)
(5, 15, 0, 0, 0)
(5, 16, 0, 0, 0)
(5, 17, 0, 0, 0)
(5, 18, 0, 0, 0)
(5, 19, 0, 0, 0)
(5, 20, 0, 0, 0)
(5, 21, 0, 0, 0)
(5, 22, 0, 0, 0)
(5, 23, 0, 0, 0)
(5, 24, 0, 0, 0)
(5, 25, 0, 0, 0)
(5, 26, 0, 0, 0)
(5, 27, 0, 0, 0)
(5, 28, 0, 0, 0)
(5, 29, 0, 0, 0)
(5, 30, 0, 0, 0)
(5, 31, 0, 0, 0)
(5, 32, 0, 0, 0)
(5, 33, 0, 0, 0)
(5, 34, 0, 0, 0)
(5, 35, 0, 0, 0)
(5, 36, 0, 0, 0)
(5, 37, 0, 0, 0)
(5, 38, 0, 0, 0)
(5, 39, 0, 0, 0)
(5, 40, 0, 0, 0)
(5, 41, 0, 0, 0)
(5, 42, 0, 0, 0)
(5, 43, 0, 0, 0)
(5, 44, 0, 0, 0)
(5, 45, 0, 0, 0)
(5, 46, 0, 0, 0)
(5, 47, 0, 0, 0)
(5, 48, 0, 0, 0)
(5, 49, 0, 0, 0)
(5, 50, 0, 0, 0)
(5, 51, 0, 0, 0)
(5, 52, 0, 0, 0)
(5, 53, 0, 0, 0)
(5, 54, 0, 0, 0)
(5, 55, 0, 0, 0)
(5, 56, 0, 0, 0)
(5, 57, 0, 0, 0)
(5, 58, 0, 0, 0)
(5, 59, 0, 0, 0)
(5, 60, 0, 0, 0)
(5, 61, 0, 0, 0)
(5, 

(5, 464, 0, 0, 0)
(5, 465, 0, 0, 0)
(5, 466, 0, 0, 0)
(5, 467, 0, 0, 0)
(5, 468, 0, 0, 0)
(5, 469, 0, 0, 0)
(5, 470, 0, 0, 0)
(5, 471, 0, 0, 0)
(5, 472, 0, 0, 0)
(5, 473, 0, 0, 0)
(5, 474, 0, 0, 0)
(5, 475, 0, 0, 0)
(5, 476, 0, 0, 0)
(5, 477, 0, 0, 0)
(5, 478, 0, 0, 0)
(5, 479, 0, 0, 0)
(5, 480, 0, 0, 0)
(5, 481, 0, 0, 0)
(5, 482, 0, 0, 0)
(5, 483, 0, 0, 0)
(5, 484, 0, 0, 0)
(5, 485, 0, 0, 0)
(5, 486, 0, 0, 0)
(5, 487, 0, 0, 0)
(5, 488, 0, 0, 0)
(5, 489, 0, 0, 0)
(5, 490, 0, 0, 0)
(5, 491, 0, 0, 0)
(5, 492, 0, 0, 0)
(5, 493, 0, 0, 0)
(5, 494, 0, 0, 0)
(5, 495, 0, 0, 0)
(5, 496, 0, 0, 0)
(5, 497, 0, 0, 0)
(5, 498, 0, 0, 0)
(5, 499, 0, 0, 0)
(5, 500, 0, 0, 0)
(5, 501, 0, 0, 0)
(5, 502, 0, 0, 0)
(5, 503, 0, 0, 0)
(5, 504, 0, 0, 0)
(5, 505, 0, 0, 0)
(5, 506, 0, 0, 0)
(5, 507, 0, 0, 0)
(5, 508, 0, 0, 0)
(5, 509, 0, 0, 0)
(5, 510, 0, 0, 0)
(5, 511, 0, 0, 0)
(5, 512, 0, 0, 0)
(5, 513, 0, 0, 0)
(5, 514, 0, 0, 0)
(5, 515, 0, 0, 0)
(5, 516, 0, 0, 0)
(5, 517, 0, 0, 0)
(5, 518, 0, 0, 0)
(5, 519, 0

(5, 920, 0, 0, 0)
(5, 921, 0, 0, 0)
(5, 922, 0, 0, 0)
(5, 923, 0, 0, 0)
(5, 924, 0, 0, 0)
(5, 925, 0, 0, 0)
(5, 926, 0, 0, 0)
(5, 927, 0, 0, 0)
(5, 928, 0, 0, 0)
(5, 929, 0, 0, 0)
(5, 930, 0, 0, 0)
(5, 931, 0, 0, 0)
(5, 932, 0, 0, 0)
(5, 933, 0, 0, 0)
(5, 934, 0, 0, 0)
(5, 935, 0, 0, 0)
(5, 936, 0, 0, 0)
(5, 937, 0, 0, 0)
(5, 938, 0, 0, 0)
(5, 939, 0, 0, 0)
(5, 940, 0, 0, 0)
(5, 941, 0, 0, 0)
(5, 942, 0, 0, 0)
(5, 943, 0, 0, 0)
(5, 944, 0, 0, 0)
(5, 945, 0, 0, 0)
(5, 946, 0, 0, 0)
(5, 947, 0, 0, 0)
(5, 948, 0, 0, 0)
(5, 949, 0, 0, 0)
(5, 950, 0, 0, 0)
(5, 951, 0, 0, 0)
(5, 952, 0, 0, 0)
(5, 953, 0, 0, 0)
(5, 954, 0, 0, 0)
(5, 955, 0, 0, 0)
(5, 956, 0, 0, 0)
(5, 957, 0, 0, 0)
(5, 958, 0, 0, 0)
(5, 959, 0, 0, 0)
(5, 960, 0, 0, 0)
(5, 961, 0, 0, 0)
(5, 962, 0, 0, 0)
(5, 963, 0, 0, 0)
(5, 964, 0, 0, 0)
(5, 965, 0, 0, 0)
(5, 966, 0, 0, 0)
(5, 967, 0, 0, 0)
(5, 968, 0, 0, 0)
(5, 969, 0, 0, 0)
(5, 970, 0, 0, 0)
(5, 971, 0, 0, 0)
(5, 972, 0, 0, 0)
(5, 973, 0, 0, 0)
(5, 974, 0, 0, 0)
(5, 975, 0